# Importing a CSV file into a PostgrSQL database

## Steps
- import a csv file into a pd dataframe 
- clean the table name and remove all the all extra symbols, spaces, capital letters
- clean the column headers and remove all the all extra symbols, spaces, capital letters
- write the create table SQL statement
- import the data into the db

In [1]:
# import libraries
import os
import numpy as np
import pandas as pd
import psycopg2
import shutil

In [ ]:
os.chdir(r'C:\-------------\World Oldest Businesses\datasets')
os.getcwd()

## Find CSV files in directory

In [3]:
# find CSV files in my currecnt working directory
# isolate only CSV files

csv_files=[]
for file in os.listdir(os.getcwd()):
    if file.endswith('.csv'):
        csv_files.append(file)

In [4]:
# make a new directory

dataset_dir = 'datasets'

# create the bash command to make a new directory
#  mkdir dataset_dir

try:
    mkdir = 'mkdir {0}'.format(dataset_dir)
    os.system(mkdir)
except:
    pass

In [5]:
# construct the full path of the file in the main folder
data_path = os.getcwd() + '\\' + dataset_dir + '\\'

In [6]:
# loop through all files in the main folder
for file_name in os.listdir(os.getcwd()):
    # check if the file is a CSV file
    if file_name.endswith('.csv'):
        # construct the full path of the file in the main folder
        file_path = os.path.join(os.getcwd(), file_name)

        # construct the full path of the file in the datasets subfolder
        new_file_path = os.path.join(data_path, file_name)

        # use the shutil module's move() function to move the file
        shutil.copy(file_path, new_file_path)

In [7]:
data_path = os.getcwd() + '/' + dataset_dir + '/'

df = {}
for file in csv_files:
    try:
        df[file] = pd.read_csv(data_path+file)
    except UnicodeDecodeError:
        df[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1")
    print(file)

businesses.csv
categories.csv
countries.csv


## Clean table names and column names

In [8]:
for k in csv_files:
    dataframe = df[k]
    
    clean_tbl_name = k.lower().replace(" ", "_").replace("-","_")\
                    .replace(r"/","_").replace("\\","_")\
                    .replace("$","").replace("%","")
    
    #remove .csv extension from clean_tbl_name
    tbl_name = '{0}'.format(clean_tbl_name.split('.')[0])
    
    dataframe.columns = [x.lower().replace(" ", "_").replace("-","_")\
                    .replace(r"/","_").replace("\\","_")\
                    .replace("$","").replace("%","") for x in dataframe.columns]   
    
    #replacment dictionary that maps pandas dtypes to sql dtypes
    replacements = {
    'object':'varchar',
    'float64':'float',
    'int64':'int',
    'datetime64':'timestamp', 
    'timedelta64[ns]':'varchar'
}    
    
    # table schema
    
    # create a list of formatted column names and data types
    columns = ["{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements))]

    # join the list into a comma-separated string
    col_str = ", ".join(columns)
    
    # open a database connection
    host='localhost'
    dbname='DataCamp'
    user='postgres'
    password='..............'


    conn_string = "host=%s dbname=%s user=%s password=%s" % (host, dbname, user, password)

    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    print('opened database successfully')
    
    # drop tables with same name
    cursor.execute('drop table if exists %s;' % (tbl_name))    
    # creat table
    cursor.execute('create table %s (%s)' % (tbl_name,col_str))
    print('{0} was created successfully'.format(tbl_name))
    
    # insert values to table

    # save the df to csv
    dataframe.to_csv(k, header=dataframe.columns, index=False, encoding='utf-8')

    # open the csv file, save it as an object
    my_file = open(k)
    print('file opened in memory')
    
    
    # upload to db
    SQL_statement = """

    COPY %s FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','

    """

    cursor.copy_expert(sql=SQL_statement % tbl_name, file=my_file)
    print('file copied to db')
    
    conn.commit()
    conn.close()
    print('table {0} imported to db completed'.format(tbl_name))
    
#for loop end message
print('all tables have been successfully imported into the db')

opened database successfully
businesses was created successfully
file opened in memory
file copied to db
table businesses imported to db completed
opened database successfully
categories was created successfully
file opened in memory
file copied to db
table categories imported to db completed
opened database successfully
countries was created successfully
file opened in memory
file copied to db
table countries imported to db completed
all tables have been successfully imported into the db
